# Test line-sink discharge

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import timml as tml

plt.rcParams["figure.figsize"] = (4, 3)

### Comparison of one line-sink with uniform strength to a row of wells with constant strength.

In [ ]:
ml1 = tml.ModelMaq(kaq=20)
rf1 = tml.Constant(ml1, xr=0, yr=20, hr=30)
ls1 = tml.LineSinkBase(ml1, x1=-10, y1=-10, x2=10, y2=10, Qls=1000)
ml1.solve()

In [ ]:
print("head at center of line-sink:", ml1.head(ls1.xc, ls1.yc))
print("discharge of line-sink:", ls1.discharge())

In [ ]:
ml2 = tml.ModelMaq(kaq=20)
rf2 = tml.Constant(ml2, xr=0, yr=20, hr=30)
N = 20
d = 20 / N
xw = np.arange(-10 + d / 2, 10, d)
yw = np.arange(-10 + d / 2, 10, d)
for i in range(N):
    tml.Well(ml2, xw[i], yw[i], Qw=1000 / N)
ml2.solve(silent=True)

In [ ]:
ml1.plots.contour(
    [-20, 20, -20, 20], 50, [0], np.arange(20, 31, 1), color="C0", labels=False
)
ml2.plots.contour(
    [-20, 20, -20, 20],
    50,
    [0],
    np.arange(20, 31, 1),
    color="C1",
    labels=False,
    newfig=False,
)

### Head line-sink of higher order with control point on the line-sink

In [ ]:
ml1 = tml.ModelMaq(kaq=[20, 10], z=[20, 12, 10, 0], c=[100])
rf1 = tml.Constant(ml1, xr=0, yr=20, hr=30)
ls1 = tml.HeadLineSink(
    ml1, -10, 0, 10, 0, 20, order=7, layers=0
)  # set control point at y=0.5
ml1.solve()

for icp in range(ls1.ncp):
    print("icp, head inside: ", icp, ls1.headinside(icp))

### Head line-sink of higher order with control point $\Delta y$ off line-sink
Useful when line-sink is used to simulate horizontal well with finite radius or ditch with finite width.
Note that this means that the head inside this distance $\Delta y$ is essentially meaningless (but a value is returned)

In [ ]:
ml1 = tml.ModelMaq(kaq=[20, 10], z=[20, 12, 10, 0], c=[100])
rf1 = tml.Constant(ml1, xr=0, yr=20, hr=30)
ls1 = tml.HeadLineSink(
    ml1, -10, 0, 10, 0, 20, order=7, layers=0, dely=0.5
)  # set control point at 0.5 from line-sink
ml1.solve()

for icp in range(ls1.ncp):
    print("icp, head inside: ", icp, ls1.headinside(icp))

### Comparison of head line-sink with row of well with specified head. 

In [ ]:
ml1 = tml.ModelMaq(kaq=[20, 10], z=[20, 12, 10, 0], c=[100])
rf1 = tml.Constant(ml1, xr=0, yr=20, hr=30)
ls1 = tml.HeadLineSink(ml1, -10, -10, 10, 10, 20, order=7, layers=0)
ml1.solve()

In [ ]:
ml2 = tml.ModelMaq(kaq=[20, 10], z=[20, 12, 10, 0], c=[100])
rf2 = tml.Constant(ml2, xr=0, yr=20, hr=30)
N = 50
d = 20 / N
xw = np.arange(-10 + d / 2, 10, d)
yw = np.arange(-10 + d / 2, 10, d)
for i in range(N):
    tml.HeadWell(ml2, xw[i], yw[i], 20, layers=0)
ml2.solve(silent=True)
Qwell = 0
for i in range(N):
    Qwell += ml2.elementlist[i + 1].discharge()

In [ ]:
print("discharge of line-sink:", ls1.discharge())
print("discharge of wells:", Qwell)

In [ ]:
ml1.plots.contour(
    [-20, 20, -20, 20], 50, [0], np.arange(20, 31, 1), color="C0", labels=False
)
ml2.plots.contour(
    [-20, 20, -20, 20],
    50,
    [0],
    np.arange(20, 31, 1),
    color="C1",
    newfig=False,
    labels=False,
)

In [ ]:
x = np.linspace(-100, 100, 100)
h1 = ml1.headalongline(x, 0)
h2 = ml2.headalongline(x, 0)
plt.figure(figsize=(8, 2))
for ilay in range(2):
    plt.plot(x, h1[ilay], label=f"line-sink layer {ilay}")
    plt.plot(x, h2[ilay], label=f"row of wells layer {ilay}")
plt.xlabel("x (m)")
plt.ylabel("head (m)")
plt.legend()
plt.grid()

## Higher-order head line-sink with resistance

In [ ]:
ml = tml.ModelMaq(kaq=3, z=[10, 0])
ls1 = tml.HeadLineSink(ml, -10, 0, 10, 0, hls=2, wh=1, res=0.2, order=2)
rf = tml.Constant(ml, 0, 20, 4)
ml.solve()

In [ ]:
print("inflow at each control-point")
for i in range(3):
    print(
        i,
        (ml.head(ls1.xc[i], ls1.yc[i]) - ls1.hc[i]) * ls1.wh / ls1.res,
        np.sum(ls1.strengthinf[i] * ls1.parameters[:, 0]),
    )

In [ ]:
print("head inside at each control point")
for i in range(3):
    print("icp, head: ", i, ls1.headinside(i))

### Higher order head line-sink with resistance in one layer of multiple layers system

In [ ]:
ml = tml.ModelMaq(kaq=[1, 2], z=[20, 10, 10, 0], c=[1000])
lslayer = 0
order = 2
ls1 = tml.HeadLineSink(
    ml, -10, 0, 10, 0, hls=-2, order=order, wh=1, res=2, layers=[lslayer]
)
rf = tml.Constant(ml, 0, 20, 2)
ml.solve()

for icp in range(ls1.ncp):
    print("icp, head inside: ", icp, ls1.headinside(icp))

In [ ]:
ml = tml.ModelMaq(kaq=[1, 2], z=[20, 12, 10, 0], c=[1000])
order = 2
ls1 = tml.HeadLineSink(
    ml, -10, 0, 10, 0, hls=-2, order=order, wh=1, res=2, layers=[0, 1]
)
rf = tml.Constant(ml, 0, 2000, 2)
ml.solve()

print("inflow at each control-point")
for i in range(order + 1):
    print(
        i,
        (ml.head(ls1.xc[i], ls1.yc[i]) - ls1.hc[i])[lslayer] * ls1.wh / ls1.res,
        np.sum(ls1.strengthinf[i] * ls1.parameters[:, 0]),
    )
print("head inside at each control point")
for icp in range(ls1.ncp):
    print("icp, head inside: ", icp, ls1.headinside(icp))

## Specifying heads along line-sinks

Give one value that is applied at all control points

In [ ]:
ml1 = tml.ModelMaq(kaq=[20, 10], z=[20, 12, 10, 0], c=[100])
rf1 = tml.Constant(ml1, xr=0, yr=20, hr=30)
ls1 = tml.HeadLineSink(ml1, -10, 0, 10, 0, hls=20, order=2, layers=[0])
ml1.solve()
print(ml1.headalongline(ls1.xc, ls1.yc))

Give `order + 1` values, which is applied at the `order + 1` control points. This may not be so useful, as the user needs to know where those control points are. 

In [ ]:
ml1 = tml.ModelMaq(kaq=[20, 10], z=[20, 12, 10, 0], c=[100])
rf1 = tml.Constant(ml1, xr=0, yr=20, hr=30)
ls1 = tml.HeadLineSink(ml1, -10, 0, 10, 0, hls=[20, 19, 18], order=2, layers=[0])
ml1.solve()
print(ml1.headalongline(ls1.xc, ls1.yc))

Give two values, the head at the beginning and end of the line-sink. Linear interpolation in between. 

In [ ]:
ml1 = tml.ModelMaq(kaq=[20, 10], z=[20, 12, 10, 0], c=[100])
rf1 = tml.Constant(ml1, xr=0, yr=20, hr=30)
ls1 = tml.HeadLineSink(ml1, -10, 0, 10, 0, hls=[19, 20], order=2, layers=[0])
ml1.solve()
print(ml1.headalongline(ls1.xc, ls1.yc))

## LineSinkDitch
Specify total discharge of ditch. Uniform head inside ditch is computed. 

Single ditch in one layer of two-aquifer system.

In [ ]:
ml1 = tml.ModelMaq(kaq=[20, 10], z=[20, 12, 10, 0], c=[100])
rf1 = tml.Constant(ml1, xr=0, yr=20, hr=30)
ls1 = tml.LineSinkDitch(ml1, -10, -10, 10, 10, Qls=1000, order=2, layers=[0])
ml1.solve()
for icp in range(ls1.ncp):
    print("icp, head inside: ", icp, ls1.headinside(icp))

Single ditch with resistance, in both layers of two-aquifer system.

In [ ]:
ml1 = tml.ModelMaq(kaq=[20, 10], z=[20, 12, 10, 0], c=[100])
rf1 = tml.Constant(ml1, xr=0, yr=20, hr=30)
ls1 = tml.LineSinkDitch(
    ml1, -10, -10, 10, 10, Qls=1000, order=2, res=0.1, layers=[0, 1]
)
ml1.solve()
for icp in range(ls1.ncp):
    print("icp, head inside: ", icp, ls1.headinside(icp))
print("discharge: ", ls1.discharge())
print("total discharge: ", np.sum(ls1.discharge()))

## Head line-sink string

In [ ]:
ml1 = tml.ModelMaq(kaq=[20, 10], z=[20, 12, 10, 0], c=[100])
rf1 = tml.Constant(ml1, xr=0, yr=20, hr=30)
ls1 = tml.HeadLineSinkString(
    ml1, xy=[(-10, 0), (0, 0), (10, 0), (10, 10)], hls=20, order=5, layers=[0]
)
ml1.solve()
ml1.plots.contour([-20, 20, -20, 20], 41, [0], 40, labels=False)
for ils in range(ls1.nls):
    print("line-sink, head inside: ", ils, ls1.lslist[ils].headinside())

Head line-sink string with linearly varying head.

In [ ]:
# linearly varying head along line-sink
ml1 = tml.ModelMaq(kaq=[20, 10], z=[20, 12, 10, 0], c=[100])
rf1 = tml.Constant(ml1, xr=0, yr=20, hr=30)
ls1 = tml.HeadLineSinkString(
    ml1, xy=[(-10, 0), (0, 0), (10, 0), (10, 10)], hls=[20, 22], order=5, layers=[0]
)
ml1.solve()
ml1.plots.contour([-20, 20, -20, 20], 41, [0], 40)
for ils in range(ls1.nls):
    print("line-sink, spec head, head inside: ", ils, ls1.lslist[ils].hc)
    for icp in range(ls1.lslist[ils].ncp):
        print("icp, head ", ls1.lslist[ils].headinside(icp))

In [ ]:
xls1 = np.linspace(-10, 10, 50)
yls1 = np.linspace(0, 0, 50)
hls1 = ml1.headalongline(xls1, yls1)
plt.figure()
plt.plot(xls1, hls1[0])
xls2 = np.linspace(10, 10, 50)
yls2 = np.linspace(0, 10, 50)
hls2 = ml1.headalongline(xls2, yls2)
plt.plot(10 + yls2, hls2[0])
plt.xlabel("length along line-sink string (m)")
plt.ylabel("head (m)")
plt.grid()

Head line-sink string with resistance.

In [ ]:
ml1 = tml.ModelMaq(kaq=[20, 10], z=[20, 12, 10, 0], c=[100])
rf1 = tml.Constant(ml1, xr=0, yr=200, hr=2)
ls1 = tml.HeadLineSinkString(
    ml1,
    xy=[(-10, 0), (0, 0), (10, 0), (10, 10)],
    hls=1,
    res=2,
    wh=5,
    order=5,
    layers=[0],
)
ml1.solve()

In [ ]:
for ils in range(ls1.nls):
    for icp in range(ls1.lslist[ils].ncp):
        print("ils, icp, head inside:", ils, icp, ls1.lslist[ils].headinside(icp))

Calculate total discharge and per linesink (using two different methods), check they
are all equal.

In [ ]:
Qtot = ls1.discharge()
Qls_sum = np.sum(ls1.discharge_per_linesink(), axis=1)
Qper_linesink_sum = np.sum([e.discharge() for e in ls1.lslist], axis=0)


assert np.allclose(Qtot, Qls_sum)
assert np.allclose(Qtot, Qper_linesink_sum)

In [ ]:
# plot head in aquifer along line-sink string
# (note there is a resistance, so this is not constant)
xls1 = np.linspace(-10, 10, 50)
yls1 = np.linspace(0, 0, 50)
hls1 = ml1.headalongline(xls1, yls1)
plt.figure()
plt.plot(xls1, hls1[0])
xls2 = np.linspace(10, 10, 50)
yls2 = np.linspace(0, 10, 50)
hls2 = ml1.headalongline(xls2, yls2)
plt.plot(10 + yls2, hls2[0])
plt.xlabel("length along string (m)")
plt.ylabel("head (m)")
plt.grid()

## Ditch string
Ditch string without resistance, screened in one layer.

In [ ]:
ml1 = tml.ModelMaq(kaq=[20, 10], z=[20, 12, 10, 0], c=[100])
rf1 = tml.Constant(ml1, xr=0, yr=20, hr=1)
ls1 = tml.LineSinkDitchString(
    ml1, xy=[(-10, 0), (0, 0), (10, 0)], Qls=100, res=0, order=5, layers=[0]
)

ml1.solve()
print("discharge:", ls1.discharge())

for ils in range(ls1.nls):
    for icp in range(ls1.lslist[ils].ncp):
        print("ils, icp, head inside:", ils, icp, ls1.lslist[ils].headinside(icp))

In [ ]:
ml1.plots.contour([-20, 20, -20, 20], 81, [0], 20, labels=False)

Ditch in multiple layers with resistance.

In [ ]:
ml1 = tml.ModelMaq(kaq=[20, 10], z=[20, 12, 10, 0], c=[100])
rf1 = tml.Constant(ml1, xr=0, yr=20, hr=1)
ls1 = tml.LineSinkDitchString(
    ml1, xy=[(-10, 0), (0, 0), (10, 0)], Qls=100, res=0.2, order=5, layers=[0, 1]
)

ml1.solve()
print("discharge:", ls1.discharge())

for ils in range(ls1.nls):
    for icp in range(ls1.lslist[ils].ncp):
        print("ils, icp, head inside:", ils, icp, ls1.lslist[ils].headinside(icp))

### Ditch in different layers

In [ ]:
ml1 = tml.ModelMaq(kaq=[20, 10], z=[20, 12, 10, 0], c=[100])
rf1 = tml.Constant(ml1, xr=0, yr=20, hr=1)
ls1 = tml.LineSinkDitchString(
    ml1,
    xy=[(-10, -10), (-5, -5), (5, 5), (10, 10)],
    Qls=100,
    wh=2,
    res=0,
    order=5,
    layers=[0, 1, 0],
)
ml1.solve()

for ils in range(ls1.nls):
    for icp in range(ls1.lslist[ils].ncp):
        print("ils, icp, head inside:", ils, icp, ls1.lslist[ils].headinside(icp))

In [ ]:
ml1.plots.contour([-20, 20, -20, 20], 81, layers=[0], levels=20, labels=False)

In [ ]:
ml1.plots.contour([-20, 20, -20, 20], 81, layers=[1], levels=20, labels=False)

### Angle well

In [ ]:
ml = tml.Model3D(
    kaq=1,
    z=np.arange(10, -0.1, -0.2),
    kzoverkh=0.1,
    topboundary="semi",
    topres=0,
    topthick=2,
    hstar=7,
)
xy = list(zip(np.linspace(-10, 10, 21), np.zeros(21), strict=False))
ls1 = tml.LineSinkDitchString(
    ml, xy=xy, Qls=100, wh=2, res=5, order=2, layers=np.arange(10, 30, 1)
)
ml.solve()

In [ ]:
ml.plots.vcontour(
    win=[-20, 20, 0, 0], n=100, levels=20, layout=True, horizontal_axis="x"
)
plt.xlabel("x (m)");